# Neural Network - Backpropagation

## Terminology

$ s_l $ : no of nodes in layer $ l $  
$ a^{(l)} $ : activation nodes for layer $ l $ , has dimension $ [m \times (s_{l}+1)] $ including the bias unit  
$ \Theta^{(l)} $ : weights for layer $ l $, has dimension $ [ (s_{l}+1) \times s_{l+1} ] $  
$ K $ : No of Output Units  
$ L $ : No of Layers

$ a^{(1)} = X = $ input layer - $ [m \times (n+1)] $   
$ \Theta^{(2)} $ - $ [s_2 \times (n+1)] $

$ a^{(2)} = g(a^{(1)}. {\Theta^{(2)}}^T ) $ - $  [m \times (s_2)] $  ( add ($ a^{(2)}_0 $) )  


and so on...

## Cost Function
$$ J(\Theta) = - \frac{1}{m} [ \sum^{m}_{i=1} \sum^{K}_{k = 1} y_{i k} log(h_{\theta} (x_i)_k ) + (1- y_{i k}) log(1- h_{\theta} (x_i)_k )] + \frac{\lambda}{2m} \sum^{L}_{l=2} \sum^{s_l}_{i=1} \sum^{s_{l+1}}_{j=1} (\Theta^{(l)}_{ij})^2  $$

## Gradient Function
For $ L = 4$ $, s_1 = 3$, $s_2 = 5$, $s_3 = 5$, $s_4 = K = 4 $  

$ \delta^{(l)}_{j} :$ "error" in the activation of node $ j$ in layer $l$  

$ \delta^{(4)} = a^{(4)} - y $ - has dimensions $[m \times s_4]$  

$ \delta^{(3)} =  \delta^{(4)} (\Theta^{(4)})^T .* g'(z^{(4)}) $ - has dimensions $[m \times s_4]$  
where, $ g'(z^{(4)}) = g(z^{(4)}) .* (1 - g(z^{(4)})) $  

Therefore,  
$ \delta^{(4)} =  \delta^{(4)} (\Theta^{(4)})^T .* a^{(4)} .* (1 - a^{(4)} )  $  

and so on...

In [229]:
import numpy as np
import scipy.optimize as op
import scipy.io as sio
import matplotlib.pyplot as plt

In [230]:
data = sio.loadmat('Practice\Machine Learning\machine-learning-ex3\ex3\ex3data1.mat')
X = data['X']
X = np.insert(arr=X, obj=0, values=1.0, axis=1)
y = data['y']
m, n = X.shape
K = 10
Y = np.zeros((m, 10))
# saving no of neurons in s
S = [400, 25, 10]   # excluding bias
# for predicting digit = 0, we get h high as index 9 (0 based)
# therefore we create Y as
# [1, 0, 0, ... 0] for 1
# [0, 1, 0, ... 0] for 2
# [0, 0, 0, ... 1] for 0
for i in range(1, K+1):
    Y[np.where(y == i)[0], i-1] = 1


In [231]:
def sigmoid(Z):
    return 1/(1+ 1/(np.e**Z))

In [253]:
# np.where( H == np.amax(H, axis=1).reshape((m, 1)) )[1]
# thetas = 3d matrix where thetas[0] = theta matrix for layer 1
# y = m x 10 shaped matrix
def costFunc(thetas, X, Y, lbd, S):
    m, n = X.shape
    Y = Y.reshape((m, S[2]))
    
    theta1 = thetas[:(S[0]+1)*S[1]]
    theta2 = thetas[(S[0]+1)*S[1]:]
    theta1 = theta1.reshape((S[1], S[0]+1))    # 25 x 401
    theta2 = theta2.reshape((S[2], S[1]+1))    # 10 x 26

    theta1 = theta1.T       # n x 25
    theta2 = theta2.T       # 26 x 10
    Z2 = X.dot(theta1)      # M x 25
    A2 = sigmoid(Z2)        
    A2 = np.insert(arr=A2, obj=0, values=1.0, axis=1)   # M x 26
    Z3 = A2.dot(theta2)
    A3 = sigmoid(Z3)        # M X 10

    cost = -1/m*np.sum(Y*np.log(A3)+(1-Y)*np.log(1-A3))
    cost += lbd /(2*m) * (np.sum(theta1[1:, :]**2) + np.sum(theta2[1:, :]**2) )
    return cost

In [254]:
test_thetas = sio.loadmat('Practice\Machine Learning\machine-learning-ex4\ex4\ex4weights.mat')
test_thetas = np.array(np.append(test_thetas['Theta1'], test_thetas['Theta2']) )
lbd = 1
cost = costFunc(test_thetas, X, Y, lbd, S)
cost

0.38376985909092365

In [255]:
def predict(thetas, X, S):
    m, n = X.shape
    
    theta1 = thetas[:(S[0]+1)*S[1]]
    theta2 = thetas[(S[0]+1)*S[1]:]
    theta1 = theta1.reshape((S[1], S[0]+1))    # 25 x 401
    theta2 = theta2.reshape((S[2], S[1]+1))    # 10 x 26

    theta1 = theta1.T       # n x 25
    theta2 = theta2.T       # 26 x 10
    Z2 = X.dot(theta1)      # M x 25
    A2 = sigmoid(Z2)        
    A2 = np.insert(arr=A2, obj=0, values=1.0, axis=1)   # M x 26
    # print(A2.shape)
    Z3 = A2.dot(theta2)
    A3 = sigmoid(Z3)        # M X 10
    # print(A3.shape)

    # choose column wise max index for each example
    # np.amax(A3, 1) gives each rows max values in an array
    # np.amax(A3, 1) gives each columns max values in an array

    # np.amax().reshape() make that a column vector
    # np.where(A3 = ... ) gives two arrays:
    # 1. Indices of rows for each match
    # 2. Indices of columns for each match
    # if an element doesn't match, it is ignored
    # we only want the indices of columns
    return np.where(A3 == np.amax(A3, axis=1).reshape((m, 1)) )[1].reshape(m, 1)+1


In [256]:
test_result = predict(test_thetas, X, S)
y[test_result==y].size/m*100

97.52

## Backpropagation

In [257]:
def sigmoidGrad(Z):
    p = sigmoid(Z)
    return p*(1-p)

In [258]:
def gradFunc(thetas, X, Y, lbd, S):
    m, n = X.shape
    Y = Y.reshape((m, S[2]))
    
    theta1 = thetas[:(S[0]+1)*S[1]]
    theta2 = thetas[(S[0]+1)*S[1]:]
    theta1 = theta1.reshape((S[1], S[0]+1))    # 25 x 401
    theta2 = theta2.reshape((S[2], S[1]+1))    # 10 x 26

    Z2 = X.dot(theta1.T)      # M x 25
    A2 = sigmoid(Z2)        
    A2 = np.insert(arr=A2, obj=0, values=1.0, axis=1)   # M x 26
    Z3 = A2.dot(theta2.T)
    A3 = sigmoid(Z3)        # M X 10

    error3 = A3 - Y     # M x 10
    temp3 = error3 * sigmoidGrad(Z3)    # M x 10
    grad3 = temp3.T.dot(A2)             # 10 * 26
    grad3[:, 1:] += (lbd/m)*theta2[:, 1:]

    error2 = temp3.dot(theta2[:, 1:])   # M x 25
    temp2 = error2 * sigmoidGrad(Z2)
    grad2 = temp2.T.dot(X)              # 25 x 401
    grad2[:, 1:] += (lbd/m)*theta1[:, 1:]

    return np.array(np.append(grad2, grad3) ).reshape((grad2.size+grad3.size, 1))

In [259]:
def randInitWeights(S):
    import numpy.random as random
    t1 = random.rand(S[1], S[0]+1)
    t2 = random.rand(S[2], S[1]+1)
    return np.array(np.append(t1, t2)).reshape((t1.size+t2.size, 1))

In [262]:
thetas = randInitWeights(S)
grad = gradFunc(test_thetas, X, Y, 1, S)
grad

array([[ 1.84729241e+00],
       [-2.11248326e-12],
       [ 4.38829369e-13],
       ...,
       [-8.86147373e-01],
       [-1.25990633e+00],
       [-9.84894986e-02]])

In [264]:
temp = op.minimize(fun=costFunc, x0=thetas,
                jac=gradFunc, args=(X, Y, 1, S), 
                method='TNC')

KeyboardInterrupt: 

In [252]:
# thetas= temp.jac.reshape((1, test_thetas.size))
temp
# theta1 = thetas[:(S[0]+1)*S[1]]
# theta2 = thetas[(S[0]+1)*S[1]:]
# theta1 = theta1.reshape((S[1], S[0]+1))    # 25 x 401
# theta2 = theta2.reshape((S[2], S[1]+1))    # 10 x 26


     fun: 37.0842319950128
     jac: array([[-1.11355333e-05],
       [ 1.16433611e-04],
       [ 4.25305774e-05],
       ...,
       [ 4.44117280e+02],
       [ 4.44117896e+02],
       [ 4.44118343e+02]])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 84
     nit: 4
  status: 1
 success: True
       x: array([ 0.15861498,  0.58216806,  0.21265289, ..., -0.19302521,
       -0.00464629, -0.10443207])

In [249]:
print(test_thetas)
predict(thetas, X, S)

[-2.25623899e-02 -1.05624163e-08  2.19414684e-09 ... -2.47795788e-01
  1.28009118e+00 -1.32752042e+00]
C:\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in power
  


array([[10],
       [10],
       [10],
       ...,
       [10],
       [10],
       [10]], dtype=int32)